# Sending raspberry pi data to AWS dynamodb table

We use a raspberry to capture temperature data and send it to a dynamo DB table through AWS IOT gateway.

- data is tranmitted in json to IOT gateway through MQTT protocol
- IOT gateway listens to a specific MQTT topic and uses a rule to push part of the message to a dynamodDB table.
- A separate jupyter notebook displays the data


![arch overview](picts/Temperature_to_iot_dynamo.png)


## Prerequisite

An AWS access and AWS CLI configured with sufficient credentials.


See 
https://viastudio.com/creating-an-aws-dynamodb-table-from-the-command-line/

## Create the dynamoDB table

### The structure of our table

Our table will have the following structure:
- deviceid (primary partition key)
- timestamp (primary sort key)
- data (MQTT message sent by the IOT thing).

We keep the default of Read and Write capacity units to the defalts (RCU: 5  WCU: 5)

#### Get info about an existing table 
Our table was previously created with web console. We can get details with the ``` describe-table``` command

In [11]:
%%bash
source ~/awscli/bin/activate
aws dynamodb describe-table --table-name=iot_data2

TABLE	1528126779.189	262	arn:aws:dynamodb:eu-west-1:814098365754:table/iot_data2	c81fdaa8-f486-4ede-978f-e08e9a04c3ff	iot_data2	34498	ACTIVE
ATTRIBUTEDEFINITIONS	deviceid	S
ATTRIBUTEDEFINITIONS	timestamp	S
KEYSCHEMA	deviceid	HASH
KEYSCHEMA	timestamp	RANGE
PROVISIONEDTHROUGHPUT	0	5	5


### Table definition in json format
we store a minimal definition in iot_data_table_def.json

In [10]:
%cat iot_data_table_def.json

{
  "AttributeDefinitions": [
    {
      "AttributeName": "deviceid",
      "AttributeType": "S"
    },
    {
      "AttributeName": "timestamp",
      "AttributeType": "S"
    }
  ],
  "TableName": "iot_data",
  "KeySchema": [
    {
      "AttributeName": "deviceid",
      "KeyType": "HASH"
    },
    {
      "AttributeName": "timestamp",
      "KeyType": "RANGE"
    }
  ],
  "ProvisionedThroughput": {
    "WriteCapacityUnits": 5,
    "ReadCapacityUnits": 5
  }
}


### Create the table in CLI

In [2]:
%%bash
source ~/awscli/bin/activate
aws dynamodb create-table --cli-input-json file://iot_data_table_def.json

TABLEDESCRIPTION	1529937491.346	0	arn:aws:dynamodb:eu-west-1:814098365754:table/iot_data	ffd35ba0-35ab-46d3-8ec6-aebf73f35b38	iot_data	0	CREATING
ATTRIBUTEDEFINITIONS	deviceid	S
ATTRIBUTEDEFINITIONS	timestamp	S
KEYSCHEMA	deviceid	HASH
KEYSCHEMA	timestamp	RANGE
PROVISIONEDTHROUGHPUT	0	5	5


## Grant access to the table to an IOT role

We create an IOT role (the IOT gateway will later impersonate this role to get wreite access to dynamo db).
A policy, that allows write on the dynamo table is then attached to this role.

### Create an IOT role  (to impersonnate iot service)
Role name: `iot_sensor_role`, its assumed policy is in `iot_sensor_role.json`

In [19]:
%cat iot_sensor_role.json

{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "iot.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}


In [24]:
%%bash
source ~/awscli/bin/activate
aws iam create-role --role-name iot_sensor_role --assume-role-policy-document file://iot_sensor_role.json

ROLE	arn:aws:iam::814098365754:role/iot_sensor_role	2018-06-25T15:47:59.316Z	/	AROAI7REHXSWLIMDGNMT4	iot_sensor_role
ASSUMEROLEPOLICYDOCUMENT	2012-10-17
STATEMENT	sts:AssumeRole	Allow
PRINCIPAL	iot.amazonaws.com


### Add a policy to allow table write access

This policy grants write access to the dynamo db table. We use the table arn (account name followed by table name): `arn:aws:dynamodb:eu-west-1:814098365754:table/iot_data`.
This arn has can be retrived from the table creation output.

In [17]:
%cat iot_table_write_policy.json

{
    "Version": "2012-10-17",
    "Statement": {
        "Effect": "Allow",
        "Action": "dynamodb:PutItem",
        "Resource": "arn:aws:dynamodb:eu-west-1:814098365754:table/iot_data"
    }
}


#### create the policy

In [26]:
%%bash
source ~/awscli/bin/activate
aws iam create-policy --policy-name iot_table_write_policy --policy-document file://iot_table_write_policy.json

POLICY	arn:aws:iam::814098365754:policy/iot_table_write_policy	0	2018-06-25T15:55:53.578Z	v1	True	/	ANPAJMEGK3TQUBEPC7QW4	iot_table_write_policy	2018-06-25T15:55:53.578Z


#### attach table write policy to iot_sensor_role

In [32]:
%%bash
source ~/awscli/bin/activate
aws iam attach-role-policy --role-name iot_sensor_role --policy-arn arn:aws:iam::814098365754:policy/iot_table_write_policy

#### Add the passrole permission

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Stmt1",
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": [
                "arn:aws:iam::814098365754:role/iot_sensor_role"
            ]
        }
    ]
}

## Register an IOT thing

Register a thing on AWS iot, we name it `lab-temp-sensor-1`.

In [33]:
%%bash
source ~/awscli/bin/activate
aws iot create-thing --thing-name=lab-temp-sensor-1

arn:aws:iot:eu-west-1:814098365754:thing/lab-temp-sensor-1	1ad6c1b2-c315-46f5-b5a4-ab82b4c93112	lab-temp-sensor-1


## Create the rule

The rule pushes messages received on the /lab/temperature topic to the dynamo db table.
Needs the correct role arn as well as table details and mapping.

In [ ]:
#### List existing rules


In [ ]:
{
  "sql": "SELECT * FROM 'iot/test'",
  "ruleDisabled": false,
  "awsIotSqlVersion": "2016-03-23",
  "actions": [{
      "dynamoDB": {
          "tableName": "my-dynamodb-table",
          "roleArn": "arn:aws:iam::814098365754:role/iot_sensor_role",
          "hashKeyField": "topic",
          "hashKeyValue": "${topic(2)}",
          "rangeKeyField": "timestamp",
          "rangeKeyValue": "${timestamp()}"
      }
  }]
}

In [ ]:
aws iot create-topic-rule --rule-name my-rule --topic-rule-payload file://my-rule.json